# Worksheet 5-1: Nonlinear Regression

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%205/Worksheet%205-1_template%20nonlinear%20regression.ipynb)

## Overview

In this worksheet, we'll explore three different approaches to fit nonlinear models to data:

1. **Linear regression on transformed data**: Converting a nonlinear model to linear form
2. **Nonlinear regression using scipy.optimize.minimize**: Direct optimization of the sum of squared residuals
3. **Nonlinear regression using scipy.optimize.curve_fit**: A more convenient wrapper for nonlinear regression

We'll apply these techniques to a bacteria growth rate model and compare their results.

## The Model

We'll work with a bacterial growth rate model that describes how the growth rate depends on substrate concentration:

$$k = k_{max} \frac{c^2}{c_s + c^2}$$

where:
- $k$ is the growth rate (number/day)
- $c$ is the substrate concentration (mg/L)
- $k_{max}$ is the maximum possible growth rate
- $c_s$ is the half-saturation constant

Our goal is to find the values of $k_{max}$ and $c_s$ that best fit our experimental data.

In [ ]:
# Import necessary libraries
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')

# Set random seed for reproducibility
np.random.seed(42)

## Experimental Data and Model

Let's start with our experimental measurements of bacterial growth rate at different substrate concentrations:

In [ ]:
# Substrate concentration (mg/L)
c = np.array([0.5, 0.8, 1.5, 2.5, 4.0])

# Growth rate (number/day)
k = np.array([1.0, 2.5, 5.1, 7.3, 9.1])

# Plot the experimental data
plt.figure(figsize=(10, 6))
plt.scatter(c, k, color='black', s=50, label='Experimental data')
plt.xlabel('Substrate Concentration (mg/L)')
plt.ylabel('Growth Rate (number/day)')
plt.title('Bacterial Growth Rate vs. Substrate Concentration')
plt.legend()
plt.grid(True)
plt.show()

# Define our model function
def kmodel(c, kmax, cs):
    """Bacterial growth rate model
    
    Parameters:
    -----------
    c : array_like
        Substrate concentration (mg/L)
    kmax : float
        Maximum growth rate (number/day)
    cs : float
        Half-saturation constant
        
    Returns:
    --------
    k : array_like
        Growth rate (number/day)
    """
    return kmax * c**2 / (cs + c**2)

## Task 1: Linear Regression with Transformed Data

Often, we can transform a nonlinear model into a linear form that allows us to use simple linear regression. For our model:

$$k = k_{max} \frac{c^2}{c_s + c^2}$$

### 1.1 Derive the Linear Transformation

In the cell below, derive a linear transformation of this model. Hint: Start by taking the reciprocal (1/k) of both sides.

**Your derivation here:**

Taking the reciprocal of both sides:

$$\frac{1}{k} = \frac{c_s + c^2}{k_{max} \cdot c^2} = \frac{c_s}{k_{max} \cdot c^2} + \frac{1}{k_{max}}$$

This is now in the form of a linear equation:

$$\frac{1}{k} = \frac{c_s}{k_{max}} \cdot \frac{1}{c^2} + \frac{1}{k_{max}}$$

Which has the form $Y = mX + b$ where:
- $Y = \frac{1}{k}$
- $X = \frac{1}{c^2}$
- $m = \frac{c_s}{k_{max}}$
- $b = \frac{1}{k_{max}}$

### 1.2 Implement the Linear Regression

Now, implement the linear regression using the transformed variables and NumPy's `polyfit` function.

In [ ]:
# Transform the data
X = 1/c**2  # X = 1/c²
Y = 1/k     # Y = 1/k

# Use np.polyfit to perform linear regression
# Complete the line below to fit a 1st-degree polynomial (straight line) to X and Y
p = np.polyfit(X, Y, 1)

# Extract the slope (m) and intercept (b)
m, b = p

# Calculate kmax and cs from m and b
# Remember: b = 1/kmax and m = cs/kmax
kmax1 = 1/b
cs1 = m * kmax1

print(f"Linear Regression Results:")
print(f"m = {m:.4f}, b = {b:.4f}")
print(f"kmax = {kmax1:.4f}")
print(f"cs = {cs1:.4f}")

## Task 2: Nonlinear Regression using scipy.optimize.minimize

Instead of transforming our model, we can directly fit the nonlinear model to the data using optimization techniques. In this task, we'll use `scipy.optimize.minimize` to minimize the sum of squared residuals.

In [ ]:
# Define the objective function to minimize (sum of squared residuals)
def objective_function(params, c_data, k_data):
    """Calculate the sum of squared residuals between model predictions and data"""
    # Extract parameters
    kmax, cs = params
    
    # Calculate predicted values
    k_pred = kmodel(c_data, kmax, cs)
    
    # Calculate residuals
    residuals = k_data - k_pred
    
    # Return sum of squared residuals
    return np.sum(residuals**2)

# Initial guess for parameters [kmax, cs]
initial_guess = [1, 1]

# Minimize the objective function
result = optimize.minimize(objective_function, 
                          initial_guess, 
                          args=(c, k), 
                          method='Nelder-Mead',
                          tol=1e-8)

# Extract the optimized parameters
kmax2, cs2 = result.x

print("Nonlinear Regression Results (minimize):")
print(f"Optimization successful: {result.success}")
print(f"Function evaluations: {result.nfev}")
print(f"kmax = {kmax2:.4f}")
print(f"cs = {cs2:.4f}")

## Task 3: Nonlinear Regression using scipy.optimize.curve_fit

The `scipy.optimize.curve_fit` function provides a more convenient interface for fitting models to data. Under the hood, it uses `scipy.optimize.minimize` but with a more user-friendly API specifically designed for curve fitting.

In [ ]:
# Use curve_fit to find the optimal parameters
popt, pcov = optimize.curve_fit(kmodel, c, k, p0=[1, 1])

# Extract the optimized parameters
kmax3, cs3 = popt

# Extract the parameter covariance matrix
parameter_variances = np.diag(pcov)
parameter_std_dev = np.sqrt(parameter_variances)

print("Nonlinear Regression Results (curve_fit):")
print(f"kmax = {kmax3:.4f} ± {parameter_std_dev[0]:.4f}")
print(f"cs = {cs3:.4f} ± {parameter_std_dev[1]:.4f}")

## Task 4: Compare All Methods

Let's define a function to calculate fit quality metrics and then compare all three methods.

In [ ]:
def calculate_fit_metrics(c_data, k_data, kmax, cs):
    """Calculate fit quality metrics for given model parameters"""
    # Calculate predicted values
    k_pred = kmodel(c_data, kmax, cs)
    
    # Calculate residuals
    residuals = k_data - k_pred
    
    # Sum of Squared Residuals (Sr)
    Sr = np.sum(residuals**2)
    
    # Total Sum of Squares (St) - variation around the mean
    St = np.sum((k_data - np.mean(k_data))**2)
    
    # Coefficient of Determination (R²)
    r_squared = 1 - Sr/St
    
    # Standard Error of the Estimate (Syx)
    Syx = np.sqrt(Sr/(len(k_data)-2))
    
    return r_squared, Syx, Sr, St

# Calculate metrics for all three methods
r_squared1, Syx_1, Sr1, St = calculate_fit_metrics(c, k, kmax1, cs1)
r_squared2, Syx_2, Sr2, St = calculate_fit_metrics(c, k, kmax2, cs2)
r_squared3, Syx_3, Sr3, St = calculate_fit_metrics(c, k, kmax3, cs3)

# Create a comparison table
methods = ['Linear Regression (transformed)', 'Nonlinear Regression (minimize)', 'Nonlinear Regression (curve_fit)']
kmax_values = [kmax1, kmax2, kmax3]
cs_values = [cs1, cs2, cs3]
r_squared_values = [r_squared1, r_squared2, r_squared3]
syx_values = [Syx_1, Syx_2, Syx_3]

# Print the comparison table
print("Comparison of Curve Fitting Methods:")
print("-" * 80)
print(f"{'Method':<30} {'kmax':<10} {'cs':<10} {'R²':<10} {'Syx':<10}")
print("-" * 80)
for i, method in enumerate(methods):
    print(f"{method:<30} {kmax_values[i]:<10.4f} {cs_values[i]:<10.4f} {r_squared_values[i]:<10.4f} {syx_values[i]:<10.4f}")
print("-" * 80)

### Visualize the Three Methods

In [ ]:
# Define a range of concentrations for plotting
c_plot = np.linspace(0, 5, 100)

# Calculate model predictions for each method
k_model1 = kmodel(c_plot, kmax1, cs1)
k_model2 = kmodel(c_plot, kmax2, cs2)
k_model3 = kmodel(c_plot, kmax3, cs3)

# Create the plot
plt.figure(figsize=(12, 8))

# Plot the data points
plt.scatter(c, k, color='black', s=80, label='Experimental data')

# Plot the model fits
plt.plot(c_plot, k_model1, 'r-', linewidth=2, 
         label=f'Linear regression: kmax={kmax1:.2f}, cs={cs1:.2f}, R²={r_squared1:.4f}')
plt.plot(c_plot, k_model2, 'b--', linewidth=2, 
         label=f'Nonlinear (minimize): kmax={kmax2:.2f}, cs={cs2:.2f}, R²={r_squared2:.4f}')
plt.plot(c_plot, k_model3, 'g-.', linewidth=2, 
         label=f'Nonlinear (curve_fit): kmax={kmax3:.2f}, cs={cs3:.2f}, R²={r_squared3:.4f}')

# Add labels and title
plt.xlabel('Substrate Concentration (mg/L)', fontsize=12)
plt.ylabel('Growth Rate (number/day)', fontsize=12)
plt.title('Comparison of Three Curve Fitting Approaches', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True)
plt.tight_layout()
plt.show()

## Discussion Questions

Answer the following questions based on your observations:

1. **Method Comparison**: Compare the results of the three methods. Which method(s) produced the best fit (highest R², lowest Syx)? Why do you think the nonlinear methods (minimize and curve_fit) gave similar results?

2. **Method Selection**: What are the advantages and disadvantages of each method? When would you prefer to use each approach for fitting nonlinear models in engineering applications?

3. **Parameter Uncertainty**: Only the `curve_fit` method provided uncertainty estimates for the parameters. Why is parameter uncertainty important in engineering analysis, and how might you use this information?